In [1]:
import os
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
from scipy.misc import imresize
from scipy import ndimage

%matplotlib inline

import random
import csv
import re
import pickle
import json
from tqdm import tqdm

DATADIR = os.path.expanduser('~/data/kaggle/ultrasound-nerve-segmentation')
PROCDIR = os.path.expanduser('~/ml/kaggle/ultrasound-nerve-segmentation/processed')
MODELDIR = os.path.expanduser('~/ml/kaggle/ultrasound-nerve-segmentation/models')

In [2]:
with open(os.path.join(PROCDIR, 'train_files_map.pkl'), 'rb') as f:
    train_files_map = pickle.load(f)
with open(os.path.join(PROCDIR, 'test_files_map.pkl'), 'rb') as f:
    test_files_map = pickle.load(f)

## create dataset

In [7]:
orig_img_rows, orig_img_cols = 420, 580
img_rows, img_cols = 128, 192
img_channels = 1

# 80/20 train/val split, by subject
data_images_all_train = []
data_presence_train = []
data_images_pos_train = []
data_masks_pos_train = []
data_images_all_val = []
data_presence_val = []
data_images_pos_val = []
data_masks_pos_val = []

subjects = set([f['subject'] for f in train_files_map.values()])
subjects_val = np.random.choice(list(subjects), int(len(subjects) * 0.2), replace=False)

for file_info in tqdm(train_files_map.values()):
    subject, img, folder, img_file, mask_file = (file_info['subject'], 
                                                 file_info['img'], 
                                                 file_info['folder'], 
                                                 file_info['img_file'], 
                                                 file_info['mask_file'])
    img_filepath = os.path.join(folder, img_file)
    mask_filepath = os.path.join(folder, mask_file)
    image = io.imread(img_filepath) / 255.0
    mask = io.imread(mask_filepath) / 255.0
    if np.sum(mask) == 0:
        presence = False
    else:
        presence = True
    image_resized = imresize(image, size=(img_rows, img_cols), interp='bilinear')
    mask_resized = imresize(mask, size=(img_rows, img_cols), interp='bilinear').astype(np.bool)
    if subject in subjects_val:
        data_images_all_val.append(image_resized)
        data_presence_val.append(presence)
        if presence:
            data_images_pos_val.append(image_resized)
            data_masks_pos_val.append(mask_resized)
    else:
        data_images_all_train.append(image_resized)
        data_presence_train.append(presence)
        if presence:
            data_images_pos_train.append(image_resized)
            data_masks_pos_train.append(mask_resized)

data_images_all_train = np.expand_dims(np.array(data_images_all_train, dtype=np.float32), axis=3)
data_presence_train = np.array(data_presence_train, dtype=np.bool)
data_images_pos_train = np.expand_dims(np.array(data_images_pos_train, dtype=np.float32), axis=3)
data_masks_pos_train = np.array(data_masks_pos_train, dtype=np.bool)
data_images_all_val = np.expand_dims(np.array(data_images_all_val, dtype=np.float32), axis=3)
data_presence_val = np.array(data_presence_val, dtype=np.bool)
data_images_pos_val = np.expand_dims(np.array(data_images_pos_val, dtype=np.float32), axis=3)
data_masks_pos_val = np.array(data_masks_pos_val, dtype=np.bool)

print('image (all) shapes:', data_images_all_train.shape, data_images_all_val.shape)
print('presence label shapes:', data_presence_train.shape, data_presence_val.shape)
print('image (pos) shapes:', data_images_pos_train.shape, data_images_pos_val.shape)
print('mask (pos) shapes:', data_masks_pos_train.shape, data_masks_pos_val.shape)

  6%|▋         | 354/5635 [00:07<01:53, 46.62it/s]/home/leon/install/miniconda3/lib/python3.5/site-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
100%|██████████| 5635/5635 [02:03<00:00, 45.67it/s]


image (all) shapes: (4555, 128, 192, 1) (1080, 128, 192, 1)
presence label shapes: (4555,) (1080,)
image (pos) shapes: (1871, 128, 192, 1) (452, 128, 192, 1)
mask (pos) shapes: (1871, 128, 192) (452, 128, 192)


## augmentation

In [8]:
multiplier = 4

data_images_all_train_augment = []
data_presence_train_augment = []
for i in tqdm(range(data_images_all_train.shape[0])):
    data_images_all_train_augment.append(data_images_all_train[i])
    data_presence_train_augment.append(data_presence_train[i])
    for j in range(multiplier):
        y_shift = random.randint(int(data_images_all_train.shape[1] * -0.125), 
                                 int(data_images_all_train.shape[1] * 0.125))
        x_shift = random.randint(int(data_images_all_train.shape[2] * -0.125), 
                                 int(data_images_all_train.shape[2] * 0.125))
        image_aug = ndimage.interpolation.shift(data_images_all_train[i], (y_shift, x_shift, 0), order=0)
        data_images_all_train_augment.append(image_aug)
        data_presence_train_augment.append(data_presence_train[i])
        
data_images_pos_train_augment = []
data_masks_pos_train_augment = []
for i in tqdm(range(data_images_pos_train.shape[0])):
    data_images_pos_train_augment.append(data_images_pos_train[i])
    data_masks_pos_train_augment.append(data_masks_pos_train[i])
    for j in range(multiplier):
        y_shift = random.randint(int(data_images_pos_train.shape[1] * -0.125), 
                                 int(data_images_pos_train.shape[1] * 0.125))
        x_shift = random.randint(int(data_images_pos_train.shape[2] * -0.125), 
                                 int(data_images_pos_train.shape[2] * 0.125))
        image_aug = ndimage.interpolation.shift(data_images_pos_train[i], (y_shift, x_shift, 0), order=0)
        mask_aug = ndimage.interpolation.shift(data_masks_pos_train[i], (y_shift, x_shift), order=0)
        data_images_pos_train_augment.append(image_aug)
        data_masks_pos_train_augment.append(mask_aug)

data_images_all_train = np.array(data_images_all_train_augment, dtype=np.float32)
data_presence_train = np.array(data_presence_train_augment, dtype=np.bool)
data_images_pos_train = np.array(data_images_pos_train_augment, dtype=np.float32)
data_masks_pos_train = np.array(data_masks_pos_train_augment, dtype=np.bool)

print('image (all) shapes:', data_images_all_train.shape, data_images_all_val.shape)
print('presence label shapes:', data_presence_train.shape, data_presence_val.shape)
print('image (pos) shapes:', data_images_pos_train.shape, data_images_pos_val.shape)
print('mask (pos) shapes:', data_masks_pos_train.shape, data_masks_pos_val.shape)

100%|██████████| 1871/1871 [00:04<00:00, 417.89it/s]


image (all) shapes: (22775, 128, 192, 1) (1080, 128, 192, 1)
presence label shapes: (22775,) (1080,)
image (pos) shapes: (9355, 128, 192, 1) (452, 128, 192, 1)
mask (pos) shapes: (9355, 128, 192) (452, 128, 192)


## save

In [9]:
with open(os.path.join(PROCDIR, 'data_train_val_08.pkl'), 'wb') as f:
    pickle.dump((data_images_all_train, data_images_all_val, 
                 data_presence_train, data_presence_val, 
                 data_images_pos_train, data_images_pos_val, 
                 data_masks_pos_train, data_masks_pos_val), f, protocol=4)